In [ ]:
# Install the yfinance api
!pip install yfinance

# Import the necessary libraries
import yfinance as yf
import pandas as pd

# Define the ticker symbol for Mercedes-Benz
ticker_symbol = "MBG.DE" 

# Create a Ticker object
mercedes = yf.Ticker(ticker_symbol)

Defaulting to user installation because normal site-packages is not writeable


### **⛏️ 1. Extracting Data**

To begin our analysis, we extracted the **Income Statement**, **Balance Sheet**, and **Cash Flow Statement** for Mercedes-Benz Group AG using the `yfinance` API.  

By leveraging the `yfinance.Ticker()` object, we were able to seamlessly download the annual financial statements for the desired period. To ensure a smooth and error-free retrieval process, we implemented robust error handling with `try` and `except` blocks, allowing the extraction to gracefully handle any unexpected issues.  

Each statement was then saved as a separate **CSV file**, making the data well-organized and easily accessible for further exploration. This modular approach ensured that we could efficiently inspect, preprocess, and analyze the financial data in the later stages of the project.


In [2]:
# Fetch Income Statement and handle errors by creating an empty DataFrame if fetch fails
try:
    income_statement = mercedes.income_stmt
    print(" Annual Income Statement : ")
    print(income_statement)
except Exception as e:
    print(f"Error fetching Income Statement: {e}")
    income_statement = pd.DataFrame() # Create an empty DataFrame if error

 Annual Income Statement : 
                                                        2024-12-31  \
Tax Effect Of Unusual Items                             43560000.0   
Tax Rate For Calcs                                           0.264   
Normalized EBITDA                                    21340000000.0   
Total Unusual Items                                    165000000.0   
Total Unusual Items Excluding Goodwill                 165000000.0   
Net Income From Continuing Operation Net Minori...   10207000000.0   
Reconciled Depreciation                               6772000000.0   
Reconciled Cost Of Revenue                          117018000000.0   
EBITDA                                               21505000000.0   
EBIT                                                 14733000000.0   
Net Interest Income                                    435000000.0   
Interest Expense                                       586000000.0   
Interest Income                                        7380000

In [ ]:
# Fetch Balance Sheet and handle errors by creating an empty DataFrame if fetch fails
try:
    balance_sheet = mercedes.balance_sheet
    print(" Annual Balance Sheet : ")
    print(balance_sheet)
except Exception as e:
    print(f"Error fetching Balance Sheet : {e}")
    balance_sheet = pd.DataFrame() # Create an empty DataFrame if error

In [ ]:
# Fetch Cash Flow and handle errors by creating an empty DataFrame if fetch fails
try:
    cash_flow = mercedes.cashflow
    print(" Annual Cash Flow Statement : ")
    print(cash_flow)
except Exception as e:
    print(f"Error fetching Cash Flow Statement: {e}")
    cash_flow = pd.DataFrame() # Create an empty DataFrame if error

In [ ]:
# Save the extracted financial statements to CSV files if they are not empty.
# This ensures that each dataset (Income Statement, Balance Sheet, Cash Flow) is stored locally for future use or backup, and provides a confirmation message for each successful save.

if not income_statement.empty:
    income_statement.to_csv("mercedes_income_statement_annual.csv")
    print("Annual Income Statement saved to mercedes_income_statement_annual.csv")
if not balance_sheet.empty:
    balance_sheet.to_csv("mercedes_balance_sheet_annual.csv")
    print("Annual Balance Sheet saved to mercedes_balance_sheet_annual.csv")
if not cash_flow.empty:
    cash_flow.to_csv("mercedes_cash_flow_annual.csv")
    print("Annual Cash Flow Statement saved to mercedes_cash_flow_annual.csv")

print("\nData acquisition complete.")

### **🔍 2. Loading & Inspecting the Extracted Data**

Before diving into the analysis, how do we ensure that our data is correctly structured and ready for use?  

With the financial datasets saved as CSV files, the first step was to load them into our workspace. We used the `pandas.read_csv()` method to import each dataset into pandas DataFrames, providing a structured and flexible format for exploration.  

Once loaded, we performed initial inspections using methods like `info()` and `head()` to get a quick overview of the data. This allowed us to verify the structure and confirm the content was correctly extracted before moving forward with cleaning and analysis.  

In [ ]:
# Load the CSV files while using the first column as the row index
income_statement_df = pd.read_csv("mercedes_income_statement_annual.csv", index_col=0)
balance_sheet_df = pd.read_csv("mercedes_balance_sheet_annual.csv", index_col=0)
cash_flow_df = pd.read_csv("mercedes_cash_flow_annual.csv", index_col=0)

print("Data loaded successfully.")

In [ ]:
# Inspecting Income Statement data
print("Income Statement Info : ")
income_statement_df.info()
print("\nFirst few rows of Income Statement:")
print(income_statement_df.head())

In [ ]:
# Inspecting Balance Sheet data
print("Balance Sheet Info : ")
balance_sheet_df.info()
print("\nFirst few rows of Balance Sheet:")
print(balance_sheet_df.head())

In [ ]:
# Inspecting Cash Flow Statement data
print("Cash Flow Statement Info : ")
cash_flow_df.info()
print("\nFirst few rows of Cash Flow Statement:")
print(cash_flow_df.head())

### **🧼 3. Data Preprocessing and Cleaning**

#### How did we prepare the data for analysis?  

Before diving into calculations and visualizations, it was essential to clean and structure the raw financial data for smooth processing. Here’s how we approached it:  


##### 1. Transposing DataFrames (`.T`)  
After inspecting the extracted DataFrames (`income_statement_df`, `balance_sheet_df`, `cash_flow_df`), we noticed that **metrics** were organized as rows and **years** as columns.  

To prepare the data for time-series analysis and ratio calculations, we transposed each DataFrame using `.T`. This step flipped the structure so that **years became the index** and **metrics became the columns**—a format that’s ideal for observing trends and performing column-wise computations.  


##### 2. Converting Index to Datetime & Sorting  
The index (which now held year-end dates such as `'2024-12-31'`) was converted from strings to proper datetime objects using `pd.to_datetime()`. This enabled us to leverage Pandas’ powerful time-series functionalities.  

We then sorted each DataFrame chronologically with `sort_index(ascending=True)` to ensure all subsequent analyses and visualizations reflected the correct temporal order.  


##### 3. Handling Missing Values (`NaN`)  
In some metrics, missing values (`NaN`) indicated **no reported data** for that year. To address this, we replaced these entries with `0` using `fillna(0)`.  

While **2020** had partial records in some statements, the data was too sparse for meaningful ratio calculations or trend analysis. We therefore excluded 2020 to ensure our insights are based on consistent and complete information.
Our analysis focuses primarily on **2021 onwards**, where data quality and consistency are much higher.  

In [ ]:
# Exclude 2020 from all three DataFrames
income_statement_df.drop('2020-12-31',axis=1,inplace=True)
balance_sheet_df.drop('2020-12-31',axis=1,inplace=True)
cash_flow_df.drop('2020-12-31',axis=1,inplace=True)

In [ ]:
#  Transpose the DataFrames
# .T is the transpose method
income_statement_transposed = income_statement_df.T
balance_sheet_transposed = balance_sheet_df.T
cash_flow_transposed = cash_flow_df.T

In [ ]:
# Convert the index to datetime objects and sort
income_statement_transposed.index = pd.to_datetime(income_statement_transposed.index)
balance_sheet_transposed.index = pd.to_datetime(balance_sheet_transposed.index)
cash_flow_transposed.index = pd.to_datetime(cash_flow_transposed.index)

income_statement_transposed = income_statement_transposed.sort_index(ascending=True)
balance_sheet_transposed = balance_sheet_transposed.sort_index(ascending=True)
cash_flow_transposed = cash_flow_transposed.sort_index(ascending=True)

In [ ]:
# Replace NaN values with 0
income_statement_transposed.fillna(0, inplace=True)
balance_sheet_transposed.fillna(0, inplace=True)
cash_flow_transposed.fillna(0, inplace=True)

### 📊 **4. Preparing Financial Data for Analysis**

##### How did we prepare the dataset for ratio calculations?  

Before diving into financial ratio analysis, it was important to focus on the most relevant metrics and align them into a single, consolidated dataset. Here’s how we approached it:  


##### 1. Selecting Relevant Metrics  
We started by narrowing down our dataset to include only the financial metrics required for ratio analysis:  

- From the transposed income statement (`income_statement_transposed`), we extracted key metrics and stored them in a DataFrame called `selected_income`.  
- Similarly, from the transposed balance sheet (`balance_sheet_transposed`) & (`cash_flow_transposed`), we selected essential metrics and saved them in `selected_balance` & `selected_cashflow` respectively.  

This ensured that we were working with a **focused dataset**, keeping only the information necessary for our analysis.  

  
##### 2. Merging DataFrames  
Next, we merged the `selected_income`, `selected_balance`, and `selected_cashflow` DataFrames into a single consolidated DataFrame named `financial_data`.  

- The merges were performed sequentially on their **date indices (years)** using `left_index=True` and `right_index=True`, ensuring all metrics aligned correctly across years.  
- This resulted in a clean, unified dataset containing all selected metrics from the income statement, balance sheet, and cash flow statement for each financial year.  
- Additionally, we initialized a new DataFrame `df` with the same date index to store the calculated financial ratios.

In [ ]:
# Displaying the columns chosen for further analysis
selected_income = income_statement_transposed[['Gross Profit', 'Net Income', 'Total Revenue','EBIT','Operating Income','Tax Rate For Calcs']]
selected_balance = balance_sheet_transposed[['Current Assets', 'Current Liabilities', 'Total Debt', 'Stockholders Equity', 'Total Assets','Invested Capital','Total Equity Gross Minority Interest']]
selected_cashflow = cash_flow_transposed[['Operating Cash Flow','Investing Cash Flow','Financing Cash Flow']]

# Merge the selected financial data into one DataFrame based on the date index
financial_data = pd.merge(selected_income, selected_balance, left_index=True, right_index=True)
financial_data = pd.merge(financial_data, selected_cashflow, left_index=True, right_index=True)

# Initialize a DataFrame to store ratios
df = pd.DataFrame(index=financial_data.index)

### 📈 **5. Feature Engineering - Calculating Financial Ratios**
After preprocessing and preparing the data, the next step was to calculate key financial ratios to extract deeper insights into the company’s performance. These ratios are crucial indicators of profitability, liquidity, solvency, and operational efficiency, helping assess the overall financial health of the business.

This brings us to the following questions:  
- How profitable is the company in relation to its revenue and investments?  
- Can the company meet its short-term obligations effectively?  
- What does the company’s capital structure reveal about its reliance on debt?  
- How efficiently is the company utilizing its assets to generate revenue?  

##### Ratio Categories :-
- **Profitability Ratios**: Gross Profit Margin, Net Profit Margin, Return on Assets (ROA), Return on Equity (ROE), and Return on Invested Capital (ROIC) measuring the company's profit generation relative to revenue and investments.
- **Liquidity Ratios**: Current Ratio, which reflects the company’s ability to meet short-term obligations.  
- **Solvency Ratios**: Debt-to-Equity Ratio, indicating the firm’s reliance on debt versus equity financing.  
- **Efficiency Ratios**: Asset Turnover Ratio, highlighting how efficiently assets are being utilized to drive revenue.  


While computing these ratios, care was taken to handle division-by-zero scenarios. Any such instances were replaced with `NaN` values using:  
`.replace([float('inf'), -float('inf')], pd.NA)`  ensuring that such instances result in NaN or pd.NA rather than errors.

Additionally, profitability and return ratios (e.g., Gross Profit Margin, Net Profit Margin, ROA, ROE, ROIC) were converted into percentages for better interpretability, in line with financial reporting conventions. Ratios such as Current Ratio, Debt-to-Equity, and Asset Turnover were retained as multipliers or decimals, reflecting standard industry practices.

In [ ]:
# 1. Gross Profit Margin
# Handle potential division by zero by setting to NaN or 0 if denominator is 0
df['Gross Profit Margin'] = (financial_data['Gross Profit'] / financial_data['Total Revenue']).replace([float('inf'), -float('inf')], pd.NA)

# 2. Net Profit Margin
df['Net Profit Margin'] = (financial_data['Net Income'] / financial_data['Total Revenue']).replace([float('inf'), -float('inf')], pd.NA)

# 3. Current Ratio
df['Current Ratio'] = (financial_data['Current Assets'] / financial_data['Current Liabilities']).replace([float('inf'), -float('inf')], pd.NA)

# 4. Debt-to-Equity Ratio
# Check for zero Stockholders Equity to avoid division by zero
df['Debt-to-Equity Ratio'] = (financial_data['Total Debt'] / financial_data['Stockholders Equity']).replace([float('inf'), -float('inf')], pd.NA)

# 5. Asset Turnover Ratio
df['Asset Turnover Ratio'] = (financial_data['Total Revenue'] / financial_data['Total Assets']).replace([float('inf'), -float('inf')], pd.NA)

# 6. Return on Equity (ROE)
df['Return on Equity (ROE)'] = (financial_data['Net Income'] / financial_data['Stockholders Equity']).replace([float('inf'), -float('inf')], pd.NA)

# 7. Return on Assets (ROA)
df['Return on Assets (ROA)'] = (financial_data['Net Income'] / financial_data['Total Assets']).replace([float('inf'), -float('inf')], pd.NA)

# 8. Return on Invested Capital (ROIC)
NOPAT = financial_data['Operating Income'] * (1 - (financial_data['Tax Rate For Calcs'] / 100))
df['Return on Invested Capital (ROIC)'] = (NOPAT / financial_data['Invested Capital']).replace([float('inf'), -float('inf')], pd.NA)


In [ ]:
# Displaying newly created dataframe for visualizations
print("Calculated Financial Ratios : ")
print(df)

print("\n Ratios Info : ")
df.info()

print("\n Missing values in Ratios DataFrame : ")
print(df.isnull().sum())

In [ ]:
# Convert ratios to percentage for better readability
df['Gross Profit Margin'] = df['Gross Profit Margin'] * 100
df['Net Profit Margin'] = df['Net Profit Margin'] * 100
df['Return on Equity (ROE)'] = df['Return on Equity (ROE)'] * 100
df['Return on Assets (ROA)'] = df['Return on Assets (ROA)'] * 100
df['Return on Invested Capital (ROIC)'] = df['Return on Invested Capital (ROIC)'] * 100
